# Computer Vision Homework 3: Big vs Small Models

## Brief

Due date: Nov 13, 2023

Required files: `homework-3.ipynb`, `report.pdf`

To download the jupyter notebook from colab, you can refer to the colab tutorial we gave.


## Codes for Problem 1 and Problem 2

### Import Packages

In [1]:
import glob
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torch.utils.data import DataLoader, Dataset, RandomSampler
from torchvision import transforms, models, datasets
from tqdm import tqdm

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


### Check GPU Environment

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device


In [3]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e6604636-90f0-a29e-7a6f-2e7f59838b5d)


### Set the Seed to Reproduce the Result

In [4]:
def set_all_seed(seed):
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
set_all_seed(123)

### Create Dataset and Dataloader

In [5]:
batch_size = 256

mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)

train_transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean, std),])

test_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(mean, std),])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transform)
valid_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

sixteenth_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//16)
half_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//2)

sixteenth_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sixteenth_train_sampler)
half_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=half_train_sampler)

100%|██████████| 170498071/170498071 [00:03<00:00, 48184715.96it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


### Load Models

In [6]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model_small = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)
model_big = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
model_small_pre = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='ResNet18_Weights.IMAGENET1K_V1')
model_big_pre = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.IMAGENET1K_V1')

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model_small.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model_small = model_small.to(device)
model_big.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model_big = model_big.to(device)
model_small_pre.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model_small_pre = model_small_pre.to(device)
model_big_pre.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model_big_pre = model_big_pre.to(device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 85.2MB/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 79.5MB/s]


### Training and Testing Models

In [7]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.

def train(dataloader, model, loss_fn, optimizer):

  num_batches = len(dataloader)
  size = len(dataloader.sampler)
  epoch_loss = 0
  correct = 0

  model.train()
  print(' * training:')

  for X, y in tqdm(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    pred = pred.argmax(dim=1, keepdim=True)
    correct += pred.eq(y.view_as(pred)).sum().item()
    #print('train correct: ', correct, 'train size: ', size)

  avg_epoch_loss = epoch_loss / num_batches
  avg_acc = correct / size

  return avg_epoch_loss, avg_acc

def test(dataloader, model, loss_fn):

  num_batches = len(dataloader)
  size = len(dataloader.sampler)
  epoch_loss = 0
  correct = 0

  model.eval()
  print(' * testing:')

  with torch.no_grad():
    for X, y in tqdm(dataloader):
      X, y = X.to(device), y.to(device)

      pred = model(X)

      epoch_loss += loss_fn(pred, y).item()
      pred = pred.argmax(dim=1, keepdim=True)
      correct += pred.eq(y.view_as(pred)).sum().item()
      #print('test correct: ', correct, 'test size: ', size)

  avg_epoch_loss = epoch_loss / num_batches
  avg_acc = correct / size

  return avg_epoch_loss, avg_acc

In [8]:
def plot_Loss_and_Accuracy(data, models_name, dataloader):
  plt.figure()
  plt.plot(data.index, data['Train_Loss'])
  plt.plot(data.index, data['Test_Loss'])
  plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}: Loss curve')
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Loss_curve.png')
  plt.close()

  plt.figure()
  plt.plot(data.index, data['Train_Acc'])
  plt.plot(data.index, data['Test_Acc'])
  plt.legend(['Train Accuracy', 'Test Accuracy'], loc='upper left')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.title(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}: Accuracy curve')
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Accuracy_curve.png')
  plt.close()

  #files.download(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Accuracy_curve.png')


In [9]:
epochs = 100
#print(f'epochs: {epochs}')
#models = [model_small, model_big]
models = [model_small_pre, model_big_pre]
#models_name = ['model_small', 'model_big']
models_name = ['model_small_pre', 'model_big_pre']
dataloaders = [sixteenth_train_dataloader, half_train_dataloader, train_dataloader]

for i, model in enumerate(models):
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  for dataloader in dataloaders:
    print(f'model: {models_name[i]}')
    print(f'dataloader\'s sampler: /{len(dataloader.dataset)//len(dataloader.sampler)}')

    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

    for epoch in range(epochs):
      print('------------------------------------------------------')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Epoch {epoch + 1:2d}:\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")

      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    locals()[models_name[i]+'_'+str(len(dataloader.dataset)//len(dataloader.sampler))+'_Train_Acc'] = train_acc
    locals()[models_name[i]+'_'+str(len(dataloader.dataset)//len(dataloader.sampler))+'_Test_Acc'] = test_acc

    plot_Loss_and_Accuracy(data, models_name[i], dataloader)
    data.to_csv(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/{models_name[i]}_{len(dataloader.dataset)//len(dataloader.sampler)}.csv')

    print("\n Done!")
    print('\n ======================================================')
'''
plt.figure()
plt.plot([1/16, 1/2, 1], [model_small_16_Train_Acc, model_small_2_Train_Acc, model_small_1_Train_Acc])
plt.plot([1/16, 1/2, 1], [model_big_16_Train_Acc, model_big_2_Train_Acc, model_big_1_Train_Acc])
plt.legend(['resnet18', 'resnet50'], loc='upper left')
plt.xlabel('Dataset Size')
plt.ylabel('Accuracy')
plt.title('Dataset Size vs Accuracy, without ImageNet initialized weights')
plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/Dataset Size vs Accuracy, without ImageNet initialized weights.png')
plt.close()
'''
plt.figure()
plt.plot([1/16, 1/2, 1], [model_small_pre_16_Train_Acc, model_small_pre_2_Train_Acc, model_small_pre_1_Train_Acc])
plt.plot([1/16, 1/2, 1], [model_big_pre_16_Train_Acc, model_big_pre_2_Train_Acc, model_big_pre_1_Train_Acc])
plt.legend(['resnet18', 'resnet50'], loc='upper left')
plt.xlabel('Dataset Size')
plt.ylabel('Accuracy')
plt.title('Dataset Size vs Accuracy, with ImageNet initialized weights')
plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/Dataset Size vs Accuracy, with ImageNet initialized weights.png')
plt.close()


model: model_small_pre
dataloader's sampler: /16
------------------------------------------------------
 * training:


100%|██████████| 13/13 [00:09<00:00,  1.36it/s]


 * testing:


100%|██████████| 40/40 [00:04<00:00,  9.49it/s]



 Epoch  1:
 Train_Loss = 1.8429, Train_Acc = 0.36 
 Test_Loss  = 1.9841, Test_Acc  = 0.35
------------------------------------------------------
 * training:


...


 Epoch 99:
 Train_Loss = 0.0604, Train_Acc = 0.98 
 Test_Loss  = 0.5896, Test_Acc  = 0.88
------------------------------------------------------
 * training:


100%|██████████| 196/196 [00:44<00:00,  4.43it/s]


 * testing:


100%|██████████| 40/40 [00:04<00:00,  9.46it/s]



 Epoch 100:
 Train_Loss = 0.0438, Train_Acc = 0.99 
 Test_Loss  = 0.5526, Test_Acc  = 0.88

 Done!



## Codes for Problem 3

In [11]:
# TODO: Try to achieve the best performance given all training data using whatever model and training strategy.
# (New) (You cannot use the model that was pretrained on CIFAR10)

class CNN(nn.Module):
  def __init__(self, num_classes: int):
    super(CNN, self).__init__()
    self.num_classes = num_classes

    self.conv1 = nn.Sequential(
        nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=5,
        stride=1,
        padding=2),
        nn.ReLU(True),
        nn.MaxPool2d(kernel_size=2))
    self.conv2 = nn.Sequential(
        nn.Conv2d(16, 32, 5, 1, 2),
        nn.ReLU(True),
        nn.MaxPool2d(2))
    self.conv3 = nn.Sequential(
        nn.Conv2d(32, 64, 5, 1, 2),
        nn.ReLU(True),
        nn.MaxPool2d(2))
    self.fc1 = nn.Sequential(
        nn.Linear(1024, 512),
        nn.ReLU(True))
    self.fc2 = nn.Sequential(
        nn.Linear(512, 256),
        nn.ReLU(True))
    self.fc3 = nn.Sequential(
        nn.Linear(256, 128),
        nn.ReLU(True))
    self.fc4 = nn.Sequential(
        nn.Linear(128, 64),
        nn.ReLU(True))
    self.fc5 = nn.Sequential(
        nn.Linear(64, 32),
        nn.ReLU(True))
    self.fc6 = nn.Sequential(
        nn.Linear(32, 16),
        nn.ReLU(True))
    self.out = nn.Linear(16, self.num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)
    x = self.fc6(x)
    output = self.out(x)
    return output


epochs = 100
cnn = CNN(10).to(torch.device('cuda'))
dataloaders = [train_dataloader]

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=3e-3)

for dataloader in dataloaders:
  print(f'dataloader\'s sampler: /{len(dataloader.dataset)//len(dataloader.sampler)}')

  data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

  for epoch in range(epochs):
    print('------------------------------------------------------')
    train_loss, train_acc = train(dataloader, cnn, loss_fn, optimizer)
    test_loss, test_acc = test(valid_dataloader, cnn, loss_fn)
    print(f"\n Epoch {epoch + 1:2d}:\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")

    data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

  plot_Loss_and_Accuracy(data, 'cnn', dataloader)
  data.to_csv(f'/content/drive/MyDrive/Colab Notebooks/計算機視覺/HW3/cnn_{len(dataloader.dataset)//len(dataloader.sampler)}.csv')

  print("\n Done!")
  print('\n ======================================================')

dataloader's sampler: /1
------------------------------------------------------
 * training:


100%|██████████| 196/196 [00:27<00:00,  7.12it/s]


 * testing:


100%|██████████| 40/40 [00:02<00:00, 15.17it/s]



 Epoch  1:
 Train_Loss = 2.0172, Train_Acc = 0.19 
 Test_Loss  = 1.8856, Test_Acc  = 0.22
------------------------------------------------------
 * training:


...


 Epoch 99:
 Train_Loss = 0.4811, Train_Acc = 0.84 
 Test_Loss  = 0.6453, Test_Acc  = 0.79
------------------------------------------------------
 * training:


100%|██████████| 196/196 [00:25<00:00,  7.71it/s]


 * testing:


100%|██████████| 40/40 [00:03<00:00, 10.78it/s]



 Epoch 100:
 Train_Loss = 0.4723, Train_Acc = 0.84 
 Test_Loss  = 0.6619, Test_Acc  = 0.79

 Done!



## Problems

1. (30%) Finish the rest of the codes for Problem 1 and Problem 2 according to the hint. (2 code cells in total.)
2. Train small model (resnet18) and big model (resnet50) from scratch on `sixteenth_train_dataloader`, `half_train_dataloader`, and `train_dataloader` respectively.
3. (30%) Achieve the best performance given all training data using whatever model and training strategy.  
  (You cannot use the model that was pretrained on CIFAR10)



## Discussion

Write down your insights in the report. The file name should be report.pdf.
For the following discussion, please present the results graphically as shown in Fig. 1 and discuss them.

- (30%) The relationship between the accuracy, model size, and the training dataset size.  
    (Total 6 models. Small model trains on the sixteenth, half, and all data. Big model trains on the sixteenth, half, and all data. If the result is different from Fig.1, please explain the possible reasons.)
- (10%) What if we train the ResNet with ImageNet initialized weights (`weights="IMAGENET1K_V1"`).
Please explain why the relationship changed this way?

Hint: You can try different hyperparameters combinations when training the models.

## Credits

1. [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)